In [2]:
# 개선된 차량 이미지 분류 파이프라인 (데이터 증강, 예측 시각화, Top-1 저장 포함)

import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import cv2
from ultralytics import YOLO

# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"[INFO] {len(gpus)} GPU(s) available: {[gpu.name for gpu in gpus]}")
else:
    print("[INFO] No GPU available. Training will use CPU.")

# 설정값
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 20,
    'LR': 2e-4,
    'BATCH_SIZE': 32,
    'SEED': 2025,
    'FOLDS': 5
}

# 시드 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# 경로 설정
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# YOLO 모델 로드
yolo_model = YOLO('yolov8n.pt')
VEHICLE_CLASSES = [2, 5, 7]

def is_full_vehicle(detections, img):
    h, w = img.shape[:2]
    img_area = h * w
    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue
        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h
        if not (0.35 * w < center_x < 0.65 * w and 0.35 * h < center_y < 0.65 * h):
            continue
        if area / img_area < 0.35:
            continue
        if aspect_ratio < 0.8 or aspect_ratio > 2.5:
            continue
        return True
    return False

# 필터링된 데이터셋 생성
os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
for cls in tqdm(os.listdir(ORIGINAL_TRAIN_DIR), desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)
    for img_path in glob(os.path.join(input_dir, '*.jpg')):
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]
        if is_full_vehicle(results.boxes, img):
            cv2.imwrite(os.path.join(output_dir, os.path.basename(img_path)), img)

# 라벨 매핑
label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# 데이터 증강 설정
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.1),
    layers.RandomBrightness(0.1)
])

def load_and_preprocess(img_path, augment=False):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    if augment:
        img = data_augmentation(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path, augment=is_train)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img
    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )
    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

def build_model(num_classes):
    base = tf.keras.applications.EfficientNetV2S(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    x = layers.Dense(512, activation='relu')(base.output)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=output)
    return model

# Stratified K-Fold 학습
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])
all_preds = []

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")
    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(train_ds, validation_data=val_ds, epochs=CFG['EPOCHS'], verbose=1)

    test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
    test_ds = create_dataset(test_paths, is_train=False)
    preds = model.predict(test_ds)
    all_preds.append(preds)

# 앙상블 평균
final_preds = np.mean(np.array(all_preds), axis=0)

# submission 저장
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = final_preds[:, idx]
submission['pred_label'] = [id2label[np.argmax(p)] for p in final_preds]
submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")

# 예측 시각화
print("예측 시각화 샘플")
for i in range(5):
    img = load_img(test_paths[i], target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    plt.imshow(img)
    pred_class = id2label[np.argmax(final_preds[i])]
    plt.title(f"예측: {pred_class}")
    plt.axis('off')
    plt.show()

[INFO] 1 GPU(s) available: ['/physical_device:GPU:0']


AttributeError: module 'torch' has no attribute '_utils'

In [3]:
!pip install tensorflow tensorflow_hub opencv-python


INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow-intel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 10.4 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   ---------------------------------------- 0/2 [tf-keras]
   -------------------